In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.trend import MACD
from tqdm import tqdm

# Ticker symbols for the 12 stocks
tickers = [
    "HSBA.L", "BARC.L", "LLOY.L", "STAN.L", "NWG.L",
    "PRU.L", "LGEN.L", "AV.L", "PHNX.L", "LSEG.L",
    "MNG.L", "HL.L"
]

# Download historical OHLCV from 2014-01-01 to 2023-12-31
def download_data(ticker):
    data = yf.download(ticker, start="2014-01-01", end="2023-12-31")
    data = data.dropna()
    return data

stock_data = {ticker: download_data(ticker) for ticker in tqdm(tickers)}

  0%|          | 0/12 [00:00<?, ?it/s]

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
100%|██████████| 12/12 [00:05<00:00,  2.01it/s]


In [2]:
def compute_features(df):
    df = df.copy()

    # Ensure Close, High, Low are Series (1D)
    close = df["Close"].squeeze()
    high = df["High"].squeeze()
    low = df["Low"].squeeze()
    volume = df["Volume"].squeeze()

    print(f"Close dtype: {type(close)}, shape: {close.shape}")

    # Daily return
    df["Return"] = close.pct_change()

    # 30-day rolling volatility (target)
    df["Volatility"] = df["Return"].rolling(window=30).std()

    # RSI (14 days)
    df["RSI"] = RSIIndicator(close=close, window=14).rsi()

    # Momentum (5 days)
    df["MOM"] = close - close.shift(5)

    # OBV
    df["OBV"] = (np.sign(close.diff()) * volume).fillna(0).cumsum()

    # MACD
    macd = MACD(close=close, window_slow=26, window_fast=12, window_sign=9)
    df["MACD_LINE"] = macd.macd()
    df["MACD_SIGNAL"] = macd.macd_signal()
    df["MACD_HIST"] = macd.macd_diff()

    # Stochastic Oscillator
    stoch = StochasticOscillator(high=high, low=low, close=close, window=14, smooth_window=3)
    df["STO_K"] = stoch.stoch()           # formerly %K
    df["STO_D"] = stoch.stoch_signal()    # formerly %D

    # Lagged volatilities (t-1 to t-6)
    for i in range(1, 7):
        df[f"Vol_t_{i}"] = df["Volatility"].shift(i)

    # Volatility t+1 (our target)
    df["Vol_target"] = df["Volatility"].shift(-1)

    # Drop rows with NaNs
    df = df.dropna()

    return df

In [3]:
import os
import pickle

feature_data = {}
for ticker in tqdm(tickers):
    feature_data[ticker] = compute_features(stock_data[ticker])

feature_data_path = "uk_feature_data.pkl"

if os.path.exists(feature_data_path):
    print("📦 Loading saved feature data from uk_feature_data.pkl...")
    with open(feature_data_path, "rb") as f:
        feature_data = pickle.load(f)
else:
    print("⚙️ Computing feature data...")
    feature_data = {ticker: compute_features(stock_data[ticker]) for ticker in tqdm(tickers)}
    with open(feature_data_path, "wb") as f:
        pickle.dump(feature_data, f)
    print("💾 Saved feature data to uk_feature_data.pkl")

100%|██████████| 12/12 [00:00<00:00, 78.28it/s]


Close dtype: <class 'pandas.core.series.Series'>, shape: (2525,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2525,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2525,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2525,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2525,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2525,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2525,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2525,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2525,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2525,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (1058,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2525,)
⚙️ Computing feature data...


  0%|          | 0/12 [00:00<?, ?it/s]

Close dtype: <class 'pandas.core.series.Series'>, shape: (2525,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2525,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2525,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2525,)


100%|██████████| 12/12 [00:00<00:00, 80.03it/s]


Close dtype: <class 'pandas.core.series.Series'>, shape: (2525,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2525,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2525,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2525,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2525,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2525,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (1058,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2525,)
💾 Saved feature data to uk_feature_data.pkl


In [4]:
from arch import arch_model
import warnings

def add_garch_predictions(df, ticker=None, verbose=True):
    df = df.copy()
    returns = df["Return"].dropna().values
    preds = []
    window_size = 500
    scale_factor = 100  # recommended by arch package

    if verbose:
        print(f"\n🔍 GARCH modeling for {ticker} — total points: {len(returns)}")

    for i in range(window_size, len(returns)):
        train_window = returns[i-window_size:i] * scale_factor  # rescale

        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                model = arch_model(train_window, vol='Garch', p=1, q=1, dist='normal', rescale=False)
                model_fit = model.fit(disp="off")
                forecast = model_fit.forecast(horizon=1)
                pred_vol_scaled = np.sqrt(forecast.variance.values[-1][0])
                pred_vol = pred_vol_scaled / scale_factor  # unscale
        except Exception as e:
            if verbose:
                print(f"⚠️ Failed at i={i} — {e}")
            pred_vol = np.nan

        preds.append(pred_vol)

        if verbose and i % 250 == 0:
            print(f"  → Index {i} | Pred Vol (unscaled): {pred_vol:.5f}")

    full_preds = [np.nan] * window_size + preds
    df["GARCH_pred"] = full_preds

    before = len(df)
    df = df.dropna()
    after = len(df)

    if verbose:
        print(f"✅ Done {ticker} | Rows dropped: {before - after} | Final: {after} rows")

    return df

In [5]:
# === Try loading precomputed garch_data from disk ===
garch_data_path = "uk_garch_data.pkl"

if os.path.exists(garch_data_path):
    print("📦 Loading saved GARCH data from uk_garch_data.pkl...")
    with open(garch_data_path, "rb") as f:
        garch_data = pickle.load(f)
    print("✅ Loaded GARCH data successfully!")
else:
    print("⚙️ Computing GARCH data from scratch...")
    garch_data = {}
    for ticker in tickers:
        print(f"\n====================== {ticker} ======================")
        garch_data[ticker] = add_garch_predictions(feature_data[ticker], ticker=ticker)

    # Save to disk
    with open(garch_data_path, "wb") as f:
        pickle.dump(garch_data, f)
    print("💾 Saved GARCH data to uk_garch_data.pkl")

⚙️ Computing GARCH data from scratch...

====================== HSBA.L ======================

🔍 GARCH modeling for HSBA.L — total points: 2488
  → Index 500 | Pred Vol (unscaled): 0.02942
  → Index 750 | Pred Vol (unscaled): 0.01212
  → Index 1000 | Pred Vol (unscaled): 0.00856
  → Index 1250 | Pred Vol (unscaled): 0.01115
  → Index 1500 | Pred Vol (unscaled): 0.01169
  → Index 1750 | Pred Vol (unscaled): 0.02449
  → Index 2000 | Pred Vol (unscaled): 0.01618
  → Index 2250 | Pred Vol (unscaled): 0.01594
✅ Done HSBA.L | Rows dropped: 500 | Final: 1988 rows

====================== BARC.L ======================

🔍 GARCH modeling for BARC.L — total points: 2488
  → Index 500 | Pred Vol (unscaled): 0.03619
  → Index 750 | Pred Vol (unscaled): 0.01737
  → Index 1000 | Pred Vol (unscaled): 0.01589
  → Index 1250 | Pred Vol (unscaled): 0.01405
  → Index 1500 | Pred Vol (unscaled): 0.01421
  → Index 1750 | Pred Vol (unscaled): 0.02405
  → Index 2000 | Pred Vol (unscaled): 0.02061
  → Index 225

In [6]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import numpy as np
import warnings
warnings.filterwarnings("ignore")

def evaluate(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "RMSE": mean_squared_error(y_true, y_pred, squared=False),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
    }

def train_ml_models_baseline(df, ticker="TICKER"):
    print(f"\n📈 Training ML models for {ticker}...")

    # Feature and target selection
    features = [
        'RSI', 'MOM', 'OBV', 'MACD_LINE', 'MACD_SIGNAL', 'MACD_HIST',
        'STO_K', 'STO_D',
        'Vol_t_1', 'Vol_t_2', 'Vol_t_3', 'Vol_t_4', 'Vol_t_5', 'Vol_t_6'
    ]

    X = df[features].copy()
    # Sanitize column names just in case LightGBM is sensitive
    X.columns = [str(col).replace("-", "_").replace("%", "PCT").replace(".", "_DOT_") for col in X.columns]

    y = df["Vol_target"]

    # Static train-test split (same as paper: 2014–2020 train, 2021–2023 test)
    split_idx = int(len(df) * 0.7)
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    models = {
        "KNN": KNeighborsRegressor(),
        "AdaBoost": AdaBoostRegressor(),
        "CatBoost": CatBoostRegressor(verbose=0),
        #"LightGBM": LGBMRegressor(),
        "XGBoost": XGBRegressor(verbosity=0),
        "RandomForest": RandomForestRegressor()
    }

    results = {}

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        metrics = evaluate(y_test, y_pred)
        results[name] = metrics
        print(f"✅ {name} — R²: {metrics['R2']:.4f}, RMSE: {metrics['RMSE']:.4f}, MAE: {metrics['MAE']:.4f}")

    return results

/shared/home/ads066/.local/lib/python3.11/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [7]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    return {
        "R2": r2_score(y_true, y_pred),
        "RMSE": np.sqrt(mse),
        "MSE": mse,
        "MAE": mean_absolute_error(y_true, y_pred),
    }

def train_all_stocks_ml_baseline(garch_data_dict, results_path="uk_ml_baseline_results.csv"):
    # Check if results already exist
    if os.path.exists(results_path):
        print(f"📦 Loading existing results from {results_path}...")
        return pd.read_csv(results_path)

    final_results = []

    for ticker, df in garch_data_dict.items():
        print(f"\n================= {ticker} =================")
        results = train_ml_models_baseline(df, ticker=ticker)

        for model_name, metrics in results.items():
            final_results.append({
                "Stock": ticker,
                "Model": model_name,
                "R2": round(metrics["R2"], 4),
                "RMSE": round(metrics["RMSE"], 4),
                "MSE": round(metrics["MSE"], 6),
                "MAE": round(metrics["MAE"], 4),
            })

    # Save results
    results_df = pd.DataFrame(final_results)
    results_df.to_csv(results_path, index=False)
    print(f"💾 Saved results to {results_path}")

    return results_df

# Run training or load existing results
ml_all_results = train_all_stocks_ml_baseline(garch_data)
ml_all_results_sorted = ml_all_results.sort_values(by="R2", ascending=False)
display(ml_all_results_sorted)


================= HSBA.L =================

📈 Training ML models for HSBA.L...
✅ KNN — R²: -1.9733, RMSE: 0.0078, MAE: 0.0065
✅ AdaBoost — R²: 0.8309, RMSE: 0.0018, MAE: 0.0014
✅ CatBoost — R²: 0.9031, RMSE: 0.0014, MAE: 0.0010
✅ XGBoost — R²: 0.8868, RMSE: 0.0015, MAE: 0.0010
✅ RandomForest — R²: 0.9020, RMSE: 0.0014, MAE: 0.0010

================= BARC.L =================

📈 Training ML models for BARC.L...
✅ KNN — R²: -6.4560, RMSE: 0.0151, MAE: 0.0124
✅ AdaBoost — R²: 0.8268, RMSE: 0.0023, MAE: 0.0017
✅ CatBoost — R²: 0.8365, RMSE: 0.0022, MAE: 0.0017
✅ XGBoost — R²: 0.8610, RMSE: 0.0021, MAE: 0.0015
✅ RandomForest — R²: 0.9131, RMSE: 0.0016, MAE: 0.0011

================= LLOY.L =================

📈 Training ML models for LLOY.L...
✅ KNN — R²: -7.1010, RMSE: 0.0171, MAE: 0.0144
✅ AdaBoost — R²: 0.8575, RMSE: 0.0023, MAE: 0.0019
✅ CatBoost — R²: 0.9306, RMSE: 0.0016, MAE: 0.0011
✅ XGBoost — R²: 0.9244, RMSE: 0.0016, MAE: 0.0011
✅ RandomForest — R²: 0.9343, RMSE: 0.0015, MAE: 0.001

,Stock,Model,R2,RMSE,MSE,MAE
33,LGEN.L,XGBoost,0.9454,0.0012,0.000001,0.0008
32,LGEN.L,CatBoost,0.9399,0.0013,0.000002,0.0009
34,LGEN.L,RandomForest,0.9355,0.0013,0.000002,0.0009
14,LLOY.L,RandomForest,0.9343,0.0015,0.000002,0.0010
12,LLOY.L,CatBoost,0.9306,0.0016,0.000002,0.0011
13,LLOY.L,XGBoost,0.9244,0.0016,0.000003,0.0011
42,PHNX.L,CatBoost,0.9180,0.0011,0.000001,0.0008
22,NWG.L,CatBoost,0.9149,0.0016,0.000002,0.0010
9,BARC.L,RandomForest,0.9131,0.0016,0.000003,0.0011
39,AV.L,RandomForest,0.9089,0.0016,0.000003,0.0010


In [8]:
from arch import arch_model
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def evaluate_series(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "RMSE": mean_squared_error(y_true, y_pred, squared=False),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
    }

def forecast_volatility_arch(df, model_type="GARCH", ticker="TICKER", verbose=True):
    df = df.copy()

    # Flatten columns if MultiIndex
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join([str(i) for i in col if i]) for col in df.columns]

    if "Vol_target" not in df.columns or "Return" not in df.columns:
        raise KeyError(f"Missing 'Vol_target' or 'Return' in {ticker}")

    returns = df["Return"].dropna().values
    preds = []
    window_size = 500
    scale_factor = 100  # fix for scale warning

    if verbose:
        print(f"\n🔮 Running {model_type} for {ticker}...")

    for i in range(window_size, len(returns)):
        train_window = returns[i-window_size:i] * scale_factor

        try:
            if model_type == "GARCH":
                model = arch_model(train_window, vol='GARCH', p=1, q=1, dist='normal', rescale=False)
            elif model_type == "GJR":
                model = arch_model(train_window, vol='GARCH', p=1, o=1, q=1, dist='normal', rescale=False)
            elif model_type == "EGARCH":
                model = arch_model(train_window, vol='EGARCH', p=1, q=1, dist='normal', rescale=False)
            else:
                raise ValueError("Invalid model_type")

            model_fit = model.fit(disp="off")
            forecast = model_fit.forecast(horizon=1)
            pred_vol = np.sqrt(forecast.variance.values[-1][0]) / scale_factor

        except Exception as e:
            if verbose:
                print(f"⚠️ {model_type} failed at index {i}: {e}")
            pred_vol = np.nan

        preds.append(pred_vol)

        if verbose and i % 250 == 0:
            print(f"  → {model_type} | index {i} | vol: {pred_vol:.5f}")

    df[f"{model_type}_pred"] = [np.nan] * window_size + preds
    df = df.dropna(subset=["Vol_target", f"{model_type}_pred"])

    metrics = evaluate_series(df["Vol_target"], df[f"{model_type}_pred"])
    if verbose:
        print(f"✅ {model_type} for {ticker} — R²: {metrics['R2']:.4f}, RMSE: {metrics['RMSE']:.4f}, MAE: {metrics['MAE']:.4f}")

    return df, metrics

In [9]:
import os
import pandas as pd

def evaluate_all_series_models(garch_data_dict, results_path="uk_ts_model_results.csv"):
    # If results already exist, load them
    if os.path.exists(results_path):
        print(f"📦 Loading saved time series results from {results_path}...")
        return pd.read_csv(results_path)

    results = []

    for ticker, df in garch_data_dict.items():
        for model_type in ["GARCH", "GJR", "EGARCH"]:
            print(f"\n================= {ticker} - {model_type} =================")
            try:
                _, metrics = forecast_volatility_arch(df, model_type=model_type, ticker=ticker, verbose=True)
                results.append({
                    "Stock": ticker,
                    "Model": model_type,
                    "R2": round(metrics["R2"], 4),
                    "RMSE": round(metrics["RMSE"], 4),
                    "MSE": round(metrics["MSE"], 6),
                    "MAE": round(metrics["MAE"], 4),
                })
            except Exception as e:
                print(f"⚠️ Skipping {ticker} - {model_type}: {e}")

    df_results = pd.DataFrame(results)
    df_results.to_csv(results_path, index=False)
    print(f"💾 Saved time series model results to {results_path}")

    return df_results

ts_model_results = evaluate_all_series_models(garch_data)
ts_model_results_sorted = ts_model_results.sort_values(by="R2", ascending=False)
display(ts_model_results_sorted)


================= HSBA.L - GARCH =================

🔮 Running GARCH for HSBA.L...
  → GARCH | index 500 | vol: 0.00856
  → GARCH | index 750 | vol: 0.01115
  → GARCH | index 1000 | vol: 0.01169
  → GARCH | index 1250 | vol: 0.02449
  → GARCH | index 1500 | vol: 0.01618
  → GARCH | index 1750 | vol: 0.01594
✅ GARCH for HSBA.L — R²: 0.8111, RMSE: 0.0028, MAE: 0.0022

================= HSBA.L - GJR =================

🔮 Running GJR for HSBA.L...
  → GJR | index 500 | vol: 0.00856
  → GJR | index 750 | vol: 0.01115
  → GJR | index 1000 | vol: 0.01166
  → GJR | index 1250 | vol: 0.02230
  → GJR | index 1500 | vol: 0.01402
  → GJR | index 1750 | vol: 0.01526
✅ GJR for HSBA.L — R²: 0.8071, RMSE: 0.0028, MAE: 0.0022

================= HSBA.L - EGARCH =================

🔮 Running EGARCH for HSBA.L...
  → EGARCH | index 500 | vol: 0.01111


/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/

  → EGARCH | index 750 | vol: 0.01150


/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/p

  → EGARCH | index 1000 | vol: 0.01335
  → EGARCH | index 1250 | vol: 0.02188
  → EGARCH | index 1500 | vol: 0.01536


...

/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


  → EGARCH | index 1750 | vol: 0.01914


/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/p

✅ EGARCH for HL.L — R²: -147.5227, RMSE: 0.1039, MAE: 0.0142
💾 Saved time series model results to uk_ts_model_results.csv


,Stock,Model,R2,RMSE,MSE,MAE
0,HSBA.L,GARCH,8.111000e-01,2.800000e-03,8.000000e-06,2.200000e-03
1,HSBA.L,GJR,8.071000e-01,2.800000e-03,8.000000e-06,2.200000e-03
22,AV.L,GJR,7.864000e-01,3.900000e-03,1.500000e-05,2.700000e-03
19,LGEN.L,GJR,7.840000e-01,5.500000e-03,3.100000e-05,3.400000e-03
18,LGEN.L,GARCH,7.703000e-01,5.700000e-03,3.200000e-05,3.300000e-03
3,BARC.L,GARCH,7.692000e-01,4.800000e-03,2.300000e-05,3.300000e-03
4,BARC.L,GJR,7.405000e-01,5.100000e-03,2.600000e-05,3.400000e-03
13,NWG.L,GJR,7.104000e-01,4.600000e-03,2.100000e-05,3.400000e-03
21,AV.L,GARCH,6.926000e-01,4.700000e-03,2.200000e-05,3.000000e-03
15,PRU.L,GARCH,6.881000e-01,5.500000e-03,3.100000e-05,4.000000e-03


In [10]:
def train_fusion_model(df, model_name, ts_feature="GARCH_pred", ticker="TICKER"):
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
    from sklearn.neighbors import KNeighborsRegressor
    from catboost import CatBoostRegressor
    from lightgbm import LGBMRegressor
    from xgboost import XGBRegressor

    models = {
        "KNN": KNeighborsRegressor(),
        "AdaBoost": AdaBoostRegressor(),
        "CatBoost": CatBoostRegressor(verbose=0),
        "XGBoost": XGBRegressor(verbosity=0),
        "RandomForest": RandomForestRegressor()
    }

    if model_name not in models:
        raise ValueError(f"Model '{model_name}' not recognized.")

    df = df.copy()

    # Flatten if needed
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join([str(i) for i in col if i]) for col in df.columns]

    if ts_feature not in df.columns:
        raise ValueError(f"'{ts_feature}' not found in DataFrame for {ticker}")

    feature_cols = [
        'RSI', 'MOM', 'OBV', 'MACD_LINE', 'MACD_SIGNAL', 'MACD_HIST',
        'STO_K', 'STO_D', 'Vol_t_1', 'Vol_t_2', 'Vol_t_3',
        'Vol_t_4', 'Vol_t_5', 'Vol_t_6', ts_feature
    ]

    X = df[feature_cols].copy()
    y = df["Vol_target"]

    split_idx = int(len(df) * 0.7)
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    model = models[model_name]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    metrics = evaluate_series(y_test, y_pred)

    print(f"✅ {model_name} + {ts_feature} for {ticker} — R²: {metrics['R2']:.4f}, RMSE: {metrics['RMSE']:.4f}, MAE: {metrics['MAE']:.4f}")

    return metrics

In [11]:
import os
import pandas as pd

# Reuse existing forecast function
for ticker in tqdm(garch_data.keys()):
    for model_type in ["GJR", "EGARCH"]:
        print(f"\n📈 Adding {model_type}_pred to {ticker}...")
        df = garch_data[ticker]

        try:
            df, _ = forecast_volatility_arch(df, model_type=model_type, ticker=ticker, verbose=False)
            garch_data[ticker] = df  # Update with new column
        except Exception as e:
            print(f"⚠️ {model_type} failed for {ticker}: {e}")

def train_all_fusion_models(garch_data_dict, results_path="uk_fusion_model_results.csv"):
    # Load existing results if file exists
    if os.path.exists(results_path):
        print(f"📦 Loading saved fusion model results from {results_path}...")
        return pd.read_csv(results_path)

    results = []

    for ticker, df in garch_data_dict.items():
        for ts_feature in ["GARCH_pred", "GJR_pred", "EGARCH_pred"]:
            if ts_feature not in df.columns:
                print(f"⚠️ Skipping {ticker} - missing {ts_feature}")
                continue

            for model_name in ["RandomForest", "XGBoost", "CatBoost", "AdaBoost", "KNN"]:
                try:
                    metrics = train_fusion_model(df, model_name, ts_feature=ts_feature, ticker=ticker)
                    results.append({
                        "Stock": ticker,
                        "Fusion_Model": f"{ts_feature}+{model_name}",
                        "R2": round(metrics["R2"], 4),
                        "RMSE": round(metrics["RMSE"], 4),
                        "MSE": round(metrics["MSE"], 6),
                        "MAE": round(metrics["MAE"], 4),
                    })
                except Exception as e:
                    print(f"⚠️ {ticker} {ts_feature}+{model_name} failed: {e}")

    fusion_df = pd.DataFrame(results)
    fusion_df.to_csv(results_path, index=False)
    print(f"💾 Saved fusion model results to {results_path}")

    return fusion_df

fusion_results_df = train_all_fusion_models(garch_data)
fusion_results_sorted = fusion_results_df.sort_values(by="R2", ascending=False)
display(fusion_results_sorted)

  0%|          | 0/12 [00:00<?, ?it/s]


📈 Adding GJR_pred to HSBA.L...

📈 Adding EGARCH_pred to HSBA.L...


/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/


📈 Adding GJR_pred to BARC.L...


/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


...

 92%|█████████▏| 11/12 [06:15<00:25, 25.19s/it]


📈 Adding EGARCH_pred to MNG.L...
⚠️ EGARCH failed for MNG.L: Length of values (500) does not match length of index (21)

📈 Adding GJR_pred to HL.L...

📈 Adding EGARCH_pred to HL.L...


/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/p

✅ RandomForest + GARCH_pred for HSBA.L — R²: 0.7989, RMSE: 0.0019, MAE: 0.0014
✅ XGBoost + GARCH_pred for HSBA.L — R²: 0.6569, RMSE: 0.0024, MAE: 0.0018
✅ CatBoost + GARCH_pred for HSBA.L — R²: 0.6906, RMSE: 0.0023, MAE: 0.0018
✅ AdaBoost + GARCH_pred for HSBA.L — R²: 0.6953, RMSE: 0.0023, MAE: 0.0019
✅ KNN + GARCH_pred for HSBA.L — R²: -6.5551, RMSE: 0.0114, MAE: 0.0100
✅ RandomForest + GJR_pred for HSBA.L — R²: 0.8385, RMSE: 0.0017, MAE: 0.0012
✅ XGBoost + GJR_pred for HSBA.L — R²: 0.7013, RMSE: 0.0023, MAE: 0.0016
✅ CatBoost + GJR_pred for HSBA.L — R²: 0.6944, RMSE: 0.0023, MAE: 0.0018
✅ AdaBoost + GJR_pred for HSBA.L — R²: 0.7411, RMSE: 0.0021, MAE: 0.0017
✅ KNN + GJR_pred for HSBA.L — R²: -6.5551, RMSE: 0.0114, MAE: 0.0100
✅ RandomForest + EGARCH_pred for HSBA.L — R²: 0.8264, RMSE: 0.0017, MAE: 0.0013
✅ XGBoost + EGARCH_pred for HSBA.L — R²: 0.6992, RMSE: 0.0023, MAE: 0.0016
✅ CatBoost + EGARCH_pred for HSBA.L — R²: 0.6937, RMSE: 0.0023, MAE: 0.0019
✅ AdaBoost + EGARCH_pred for HS

,Stock,Fusion_Model,R2,RMSE,MSE,MAE
15,BARC.L,GARCH_pred+RandomForest,0.9193,0.0014,0.000002,0.0010
25,BARC.L,EGARCH_pred+RandomForest,0.9162,0.0014,0.000002,0.0010
20,BARC.L,GJR_pred+RandomForest,0.9144,0.0014,0.000002,0.0010
80,PRU.L,GJR_pred+RandomForest,0.9133,0.0017,0.000003,0.0011
30,LLOY.L,GARCH_pred+RandomForest,0.9125,0.0013,0.000002,0.0010
...,...,...,...,...,...,...
9,HSBA.L,GJR_pred+KNN,-6.5551,0.0114,0.000131,0.0100
4,HSBA.L,GARCH_pred+KNN,-6.5551,0.0114,0.000131,0.0100
104,LGEN.L,EGARCH_pred+KNN,-55.8425,0.0317,0.001007,0.0280
99,LGEN.L,GJR_pred+KNN,-55.8425,0.0317,0.001007,0.0280
